In [1]:
#final
#final bototrch model

##federated learning with 2 clients

import torch
import math
#import sys

#sys.path.append('/content/drive/My Drive')
from utilities import find_ate, find_ate_2, find_statistical_parity_score, find_eqop_score, all_metrics
from load_data_redefined_ate import get_data, load_dataset
from constraint import AverageTreatmentEffectLoss, DemographicParityLoss, EqualOpportunityLoss
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score

from torch import nn
from torch import optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

from torch.utils.data import DataLoader
from torch import nn, optim
from torch.utils.data import TensorDataset
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from botorch.models import SingleTaskGP, ModelListGP
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch.fit import fit_gpytorch_model
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import qExpectedImprovement
from botorch.optim import optimize_acqf
from itertools import chain
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.acquisition.multi_objective.monte_carlo import (
    qExpectedHypervolumeImprovement,
    qNoisyExpectedHypervolumeImprovement,
)
from botorch.acquisition.multi_objective.objective import IdentityMCMultiOutputObjective
from botorch import fit_gpytorch_mll
import math
from botorch.utils.transforms import unnormalize, normalize
from sklearn.metrics import confusion_matrix
from botorch.utils.multi_objective.hypervolume import Hypervolume
from botorch.utils.multi_objective.pareto import is_non_dominated
from botorch.utils.multi_objective.box_decompositions.non_dominated import (
    FastNondominatedPartitioning,
)
device = torch.device('mps')
'''
#logistic regression model
def create_model(input_dim):
    model = nn.Sequential(
        nn.Linear(input_dim, 1),
        nn.Sigmoid()
    )
    return model
'''
#DNN model
def create_model(input_dim):
    model = nn.Sequential(
        nn.Linear(input_dim, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, 1),
        nn.Sigmoid()
    )
    return model

# set parameters
fairness_notion = 'stat_parity' #'ate' 'stat_parity'
num_clients = 3
dataset_name = 'adult'
epochs = 15 #client training epochs
communication_rounds = 50

if dataset_name == 'adult':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/adult.csv'
    sensitive_feature = 'sex' #'sex': 0 ->female, 1-> male
elif dataset_name == 'adult-age':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/adult.csv'
    sensitive_feature = 'sex' #'sex': 0 ->female, 1-> male
elif dataset_name == 'bank':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/bank-full.csv'
    sensitive_feature = 'marital' #'marital': 0->married, 1-> single
elif dataset_name == 'bank-age':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/bank-full.csv'
    sensitive_feature = 'marital' #'marital': 0->married, 1-> single
elif dataset_name == 'default':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/default.csv'
    sensitive_feature = 'SEX' #'sex': 0 ->female, 1-> male
elif dataset_name == 'default-age':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/default.csv'
    sensitive_feature = 'SEX' #'sex': 0 ->female, 1-> male
elif dataset_name == 'law':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/law.csv'
    sensitive_feature = 'sex' #'sex': 0 ->female, 1-> male
elif dataset_name == 'kdd':
    url = './kdd.csv'
    sensitive_feature = 'sex' #'sex': 0 ->female, 1-> male
elif dataset_name == 'kdd-age':
    url = './kdd.csv'
    sensitive_feature = 'sex' #'sex': 0 ->female, 1-> male
elif dataset_name == 'law-income':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/law.csv'
    sensitive_feature = 'sex' #'sex': 0 ->female, 1-> male
else:
    print("dataset not supported, please update file load_data.py")
    exit()

bal_acc_list = []
fairness_notion_list = []
clients_data,X_test, y_test, sex_list, column_names_list, ytest_potential = load_dataset(url,dataset_name, num_clients, sensitive_feature)
X_test = X_test.to(device)
y_test = y_test.to(device)
global_model = create_model(X_test.shape[1])
global_model = global_model.to(device)

# Define a loss function and optimizer
criterion = nn.BCELoss()

def initialize_model(train_x, train_y):
    models = []
    for i in range(train_y.shape[-1]):
        train_objective = train_y[:,i]
        print("bismillah")
        models.append(
          SingleTaskGP(train_x, train_objective.unsqueeze(-1))
        )
    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
    return model,mll


cost_false_negatives = 10.0 #15 for adult-age
cost_false_positives = 1.0

def calculate_weights(targets, cost_false_negatives=5):
    cost_false_negatives = 10
    # Give higher weight to the positive samples because false negatives cost more
    return torch.where(targets == 1, cost_false_negatives, cost_false_positives)

def evaluate(alpha = 100, lr=0.001, cost_false_negatives=5):
    # Initialize a list to store the parameters of each model
    params = [torch.zeros_like(param.data) for param in global_model.parameters()]
    for client_name in clients_data.keys():
        print(client_name)
        X1,y1,s1,y1_potential = get_data(client_name, clients_data)
        X1 = X1.to(device)
        y1 = y1.to(device)
        y1_potential = y1_potential.to(device)
        s1 = s1.to(device)
        model1 = create_model(X1.shape[1])
        model1 = model1.to(device)
        model1.load_state_dict(global_model.state_dict())
        optimizer1 = optim.Adam(model1.parameters(), lr=lr)
        if fairness_notion == 'stat_parity':
            dp_loss = DemographicParityLoss(alpha=alpha)
        elif fairness_notion == 'ate':
            dp_loss = EqualOpportunityLoss(alpha=alpha)
        #dp_loss = AverageTreatmentEffectLoss(alpha=alpha)
        for epoch in range(epochs):
            criterion = nn.BCEWithLogitsLoss(pos_weight=None)
            # Training on Client 1
            optimizer1.zero_grad()
            y_pred = model1(X1)
            #class_weights = find_class_weights(y1)
            #criterion = nn.BCELoss(weight=class_weights)
            weights = calculate_weights(y1,cost_false_negatives)
            criterion = torch.nn.BCEWithLogitsLoss(pos_weight=weights)
            X1_cpu = X1.cpu()
            X1_dataframe = pd.DataFrame(X1_cpu.numpy(), columns=column_names_list)
            y_pred_numpy = y_pred.clone().cpu()
            
            #y1_potential = find_potential_outcomes(X1_dataframe,y_pred_numpy.round().detach().numpy())
            #y1_potential = torch.tensor(y1_potential, dtype=torch.float32)
            fairness_loss = dp_loss(X1, y_pred, s1,y1_potential)
            fairness_loss = fairness_loss.to(device)
            loss = criterion(y_pred.view(-1), y1) + fairness_loss
            loss.backward()
            optimizer1.step()
        print(f'- Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')
        # After training, add the trained model's parameters to the list
        
        for param, param_sum in zip(model1.parameters(), params):
            param_sum.add_(param.data)

    # Compute average of parameters and update global model
    average_params = [param_sum / len(clients_data) for param_sum in params]
    # Copy the averaged parameters into the global model
    with torch.no_grad():
        for param_global, param_avg in zip(global_model.parameters(), average_params):
            param_global.copy_(param_avg)
    global_model.eval()
    
    # Average the model parameters (weights and biases) and set the averaged parameters to both models
    with torch.no_grad():
        y_pred = global_model(X_test).squeeze()
        y_pred_cls = y_pred.round()
        sensitivity,specificity,bal_acc,G_mean,FN_rate,FP_rate,Precision,f1_sc, acc, auc = all_metrics(y_test.cpu(),y_pred.cpu())
        stat_parity = find_statistical_parity_score(sex_list, y_test,y_pred_cls)
        X_test_cpu = X_test.cpu()
        Xtest_dataframe = pd.DataFrame(X_test_cpu.numpy(), columns=column_names_list)
        y_pred_numpy = y_pred.clone().cpu()
        #ytest_potential = find_potential_outcomes(Xtest_dataframe,y_pred_numpy.round().detach().numpy())
        ate = find_ate_2(y_pred_cls.cpu(), ytest_potential, sex_list)#0 means female-protected attribute
        #acc = (y_pred_cls == y_test).float().mean()
        auprc = average_precision_score(y_test.cpu(), y_pred.cpu())
        print(f'Communication round {round+1}/{communication_rounds}')
        if communication_rounds % 1 == 0:
            print(f'Test accuracy: {acc.item()}')
            print("sensitivity: %s" % sensitivity)
            print("specificity: %s" % specificity)
            print("BalanceACC: %s" % bal_acc)
            print("G_mean: %s" % G_mean)
            print("FN_rate: %s" % FN_rate)
            print("FP_rate: %s" % FP_rate)
            print("Precision: %s" % Precision)
            print("f1_sc: %s" % f1_sc)
            print("statistical parity: %s" % stat_parity)
            print("ate: %s" % ate)
            print("auc: %s" % auc)
            print("auprc: %s" % auprc)
    if fairness_notion == 'stat_parity':
        objectives = torch.tensor([[-stat_parity, bal_acc]]) #the two objectives
    elif fairness_notion == 'ate':
        objectives = torch.tensor([[-ate, bal_acc]]) #the two objectives
    return objectives

def models_have_same_parameters(model1, model2):
    params1 = list(model1.parameters())
    params2 = list(model2.parameters())
    print(params1)
    print("bismillah")
    print(params2)
    if len(params1) != len(params2):
        return False
    
    for p1, p2 in zip(params1, params2):
        if not torch.allclose(p1, p2):
            return False

    return True


# Train the model on both clients

for round in range(communication_rounds):
    
    print(f'Communication round {round+1}/{communication_rounds}')

    bounds = torch.tensor([[100.0,0.0], [2000.0,0.01]])  # bounds on learning rate
    #bounds = torch.tensor([[0.0], [350.0]])
    alpha = torch.tensor([100], dtype=torch.float32)
    alpha = alpha.view(1, -1)

    objectives = evaluate(alpha)
    fairness_notion_list.append(objectives[0,0].item())
    bal_acc_list.append(objectives[0,1].item())
    
    #alpha = normalize(alpha, bounds)
    x_input =  torch.tensor([100,0.001], dtype=torch.float32)#input to the optimization process
    x_input = x_input.view(1, -1)
    #model, mll = initialize_model(alpha, objectives.float())
    model, mll = initialize_model(x_input, objectives.float())

    for i in range(10):  # number of rounds of optimization
        print("Global optimization round:", i)
        fit_gpytorch_mll(mll)
        ref_point=torch.tensor([0.0001, 0.001])
        acq_func = qExpectedHypervolumeImprovement(
            model=model.float(),
            ref_point=torch.tensor([0.001, 0.001]),#problem.ref_point.tolist(),  # use known reference point
            sampler=SobolQMCNormalSampler(sample_shape=torch.Size([128])),
            # define an objective that specifies which outcomes are the objectives
            objective=IdentityMCMultiOutputObjective(outcomes=[0, 1]),
            partitioning = FastNondominatedPartitioning(ref_point, Y = objectives)
            
            # specify that the constraint is on the last outcome
            #constraints=[lambda Z: Z[..., -1]],
        )
        candidate, acq_value = optimize_acqf(
            acq_function=acq_func,
            bounds=bounds,
            q=1,
            num_restarts=300,
            raw_samples=1024,
            options ={"batch_limit": 5, "maxiter": 200}
        )


        #new_candidate = unnormalize(candidate, bounds)
        for i, m in enumerate(model.models):
          # Iterate over models and update the train data for each

            new_objectives = evaluate(candidate[0,0].item(), candidate[0,1].item())#evaluate(candidate.item())

            for i, m in enumerate(model.models):
                train_x = torch.cat([m.train_inputs[0], candidate])
                print("bismillah")
                train_y = torch.cat([m.train_targets, new_objectives[:,i]])
                m.set_train_data(train_x, train_y, strict=False)

    # Now both models have the same parameters and we can continue with the next round of communication


global_model.eval()
    
# Average the model parameters (weights and biases) and set the averaged parameters to both models
with torch.no_grad():
        y_pred = global_model(X_test).squeeze()
        y_pred_cls = y_pred.round()
        sensitivity,specificity,bal_acc,G_mean,FN_rate,FP_rate,Precision,f1_sc, acc, auc = all_metrics(y_test.cpu(),y_pred.cpu())
        stat_parity = find_statistical_parity_score(sex_list, y_test,y_pred_cls)
        X_test_cpu = X_test.cpu()
        Xtest_dataframe = pd.DataFrame(X_test_cpu.numpy(), columns=column_names_list)
        y_pred_numpy = y_pred.clone().cpu()
        #ytest_potential = find_potential_outcomes(Xtest_dataframe,y_pred_numpy.round().detach().numpy())
        ate = find_ate_2(y_pred_cls.cpu(), ytest_potential, sex_list)#0 means female-protected attribute
        #acc = (y_pred_cls == y_test).float().mean()
        auprc = average_precision_score(y_test.cpu(), y_pred.cpu())
        print(f'Test accuracy: {acc.item()}')
        print("sensitivity: %s" % sensitivity)
        print("specificity: %s" % specificity)
        print("BalanceACC: %s" % bal_acc)
        print("G_mean: %s" % G_mean)
        print("FN_rate: %s" % FN_rate)
        print("FP_rate: %s" % FP_rate)
        print("Precision: %s" % Precision)
        print("f1_sc: %s" % f1_sc)
        print("statistical parity: %s" % stat_parity)
        print("ate: %s" % ate)
        print("auc: %s" % auc)
        print("auprc: %s" % auprc)


destination = './results/regression/adult/'

if dataset_name == 'adult' or dataset_name == 'bank' or dataset_name == 'law'or dataset_name == 'default'or dataset_name == 'kdd':
    if fairness_notion == 'stat_parity':
        np.save(destination+str(num_clients)+'_bal_acc_stat_parity.npy', np.array(bal_acc_list))
        np.save(destination+str(num_clients)+'_stat_parity.npy', np.array(fairness_notion_list))
    else:
        np.save(destination+str(num_clients)+'_bal_acc_ate.npy', np.array(bal_acc_list))
        np.save(destination+str(num_clients)+'_ate.npy', np.array(fairness_notion_list))
else:
    if fairness_notion == 'stat_parity':
        np.save(destination+str(num_clients)+'_attr_bal_acc_stat_parity.npy', np.array(bal_acc_list))
        np.save(destination+str(num_clients)+'_attr_stat_parity.npy', np.array(fairness_notion_list))
    else:
        np.save(destination+str(num_clients)+'_attr_bal_acc_ate.npy', np.array(bal_acc_list))
        np.save(destination+str(num_clients)+'_attr_ate.npy', np.array(fairness_notion_list))


/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Communication round 1/50
client_1
- Epoch 15/15, Loss: 1.8863039016723633
client_2
- Epoch 15/15, Loss: 1.890197992324829
client_3
- Epoch 15/15, Loss: 1.8727167844772339
Communication round 1/50
Test accuracy: 0.2855826807922616
sensitivity: 0.8243331164606376
specificity: 0.11917202572347267
BalanceACC: 0.4717525710920551
G_mean: 0.3134285362878714
FN_rate: 0.17566688353936238
FP_rate: 0.8808279742765274
Precision: 0.22424778761061948
f1_sc: 0.35258104911646027
statistical parity: 0.08514522087805798
ate: -0.006634650308729173
auc: 0.3844955460903292
auprc: 0.1805358969927442
bismillah
bismillah
Global optimization round: 0


/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/botorch/models/gpytorch.py:129: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  warnings.warn(_get_single_precision_warning(X.dtype), UserWarning)
/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/botorch/models/utils/assorted.py:173: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)
/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/botorch/models/utils/assorted.py:201: InputDataWarning: Input data is not standardized. Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


client_1
- Epoch 15/15, Loss: 1.8644874095916748
client_2
- Epoch 15/15, Loss: 1.8719335794448853
client_3
- Epoch 15/15, Loss: 1.856287956237793
Communication round 1/50
Test accuracy: 0.25011515430677106
sensitivity: 0.9102147039687704
specificity: 0.04622186495176849
BalanceACC: 0.4782182844602694
G_mean: 0.20511416607333205
FN_rate: 0.08978529603122967
FP_rate: 0.9537781350482315
Precision: 0.2276647681041497
f1_sc: 0.36422806560791454
statistical parity: 0.030771443302922696
ate: -0.0008292235267575165
auc: 0.3867182515109611
auprc: 0.18114630294844847
bismillah
bismillah
client_1
- Epoch 15/15, Loss: 1.8337517976760864
client_2
- Epoch 15/15, Loss: 1.8391780853271484
client_3
- Epoch 15/15, Loss: 1.8259066343307495
Communication round 1/50
Test accuracy: 0.24028865346230616
sensitivity: 0.9648666232921275
specificity: 0.01647909967845659
BalanceACC: 0.49067286148529204
G_mean: 0.12609573054487924
FN_rate: 0.03513337670787248
FP_rate: 0.9835209003215434
Precision: 0.23255449270817

/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/botorch/optim/fit.py:130: OptimizationWarning: `scipy_minimize` terminated with status 3, displaying original message from `scipy.optimize.minimize`: ABNORMAL_TERMINATION_IN_LNSRCH
  warn(


...

/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/botorch/optim/optimize.py:369: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.')]
Trying again with a new set of initial conditions.
  warnings.warn(first_warn_msg, RuntimeWarning)


client_1
- Epoch 15/15, Loss: 1.5709182024002075
client_2
- Epoch 15/15, Loss: 1.569532036781311
client_3
- Epoch 15/15, Loss: 1.5647820234298706
Communication round 49/50
Test accuracy: 0.6152310763089206
sensitivity: 0.9355888093689004
specificity: 0.5162781350482315
BalanceACC: 0.725933472208566
G_mean: 0.6949993134334532
FN_rate: 0.06441119063109954
FP_rate: 0.4837218649517685
Precision: 0.3739921976592978
f1_sc: 0.5343738387216649
statistical parity: 0.002109388649079391
ate: -0.009257763104580818
auc: 0.7984334565184192
auprc: 0.47314392286573986
bismillah
bismillah
client_1
- Epoch 15/15, Loss: 1.570961833000183
client_2
- Epoch 15/15, Loss: 1.5694876909255981
client_3
- Epoch 15/15, Loss: 1.5648448467254639
Communication round 49/50
Test accuracy: 0.615691693536005
sensitivity: 0.9355888093689004
specificity: 0.5168810289389068
BalanceACC: 0.7262349191539036
G_mean: 0.6954049945537665
FN_rate: 0.06441119063109954
FP_rate: 0.48311897106109325
Precision: 0.3742842269651223
f1_sc:

/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/botorch/models/gpytorch.py:129: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  warnings.warn(_get_single_precision_warning(X.dtype), UserWarning)
/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/botorch/models/utils/assorted.py:173: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)
/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/botorch/models/utils/assorted.py:201: InputDataWarning: Input data is not standardized. Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


client_1
- Epoch 15/15, Loss: 1.5708755254745483
client_2
- Epoch 15/15, Loss: 1.5692890882492065
client_3
- Epoch 15/15, Loss: 1.5646644830703735
Communication round 50/50
Test accuracy: 0.6178412405957316
sensitivity: 0.9349381912817176
specificity: 0.519895498392283
BalanceACC: 0.7274168448370003
G_mean: 0.6971873183889593
FN_rate: 0.06506180871828236
FP_rate: 0.48010450160771706
Precision: 0.3755880815473079
f1_sc: 0.5358940891292188
statistical parity: 0.0053836762869816646
ate: -0.007590966647498032
auc: 0.7984148898447094
auprc: 0.4730842672176159
bismillah
bismillah
client_1
- Epoch 15/15, Loss: 1.5708270072937012
client_2
- Epoch 15/15, Loss: 1.5693832635879517
client_3
- Epoch 15/15, Loss: 1.5646775960922241
Communication round 50/50
Test accuracy: 0.6175341624443421
sensitivity: 0.9349381912817176
specificity: 0.5194935691318328
BalanceACC: 0.7272158802067752
G_mean: 0.6969177698312762
FN_rate: 0.06506180871828236
FP_rate: 0.4805064308681672
Precision: 0.37539184952978055
f1

client_1
- Epoch 15/15, Loss: 1.57064688205719
client_2
- Epoch 15/15, Loss: 1.5693200826644897
client_3
- Epoch 15/15, Loss: 1.5647231340408325
Communication round 50/50
Test accuracy: 0.6173806233686473
sensitivity: 0.9342875731945348
specificity: 0.5194935691318328
BalanceACC: 0.7268905711631838
G_mean: 0.6966752371043106
FN_rate: 0.06571242680546518
FP_rate: 0.4805064308681672
Precision: 0.37522863862032924
f1_sc: 0.5354213273676361
statistical parity: 0.004919208103052242
ate: -0.007828449886472022
auc: 0.7984265267035839
auprc: 0.47310364346003464
bismillah
bismillah
client_1
- Epoch 15/15, Loss: 1.5706344842910767
client_2
- Epoch 15/15, Loss: 1.5693470239639282
client_3
- Epoch 15/15, Loss: 1.5647380352020264
Communication round 50/50
Test accuracy: 0.6183018578228159
sensitivity: 0.9342875731945348
specificity: 0.5206993569131833
BalanceACC: 0.727493465053859
G_mean: 0.697483289071769
FN_rate: 0.06571242680546518
FP_rate: 0.47930064308681675
Precision: 0.3758178487306988
f1_sc

In [1]:
#final bototrch model

##federated learning with 2 clients

import torch
import math
#import sys

#sys.path.append('/content/drive/My Drive')
from utilities import find_ate, find_ate_2, find_statistical_parity_score, find_eqop_score, all_metrics
from load_data_redefined_ate import get_data, load_dataset
from constraint import AverageTreatmentEffectLoss, DemographicParityLoss, EqualOpportunityLoss
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score

from torch import nn
from torch import optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

from torch.utils.data import DataLoader
from torch import nn, optim
from torch.utils.data import TensorDataset
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from botorch.models import SingleTaskGP, ModelListGP
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch.fit import fit_gpytorch_model
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import qExpectedImprovement
from botorch.optim import optimize_acqf
from itertools import chain
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.acquisition.multi_objective.monte_carlo import (
    qExpectedHypervolumeImprovement,
    qNoisyExpectedHypervolumeImprovement,
)
from botorch.acquisition.multi_objective.objective import IdentityMCMultiOutputObjective
from botorch import fit_gpytorch_mll
import math
from botorch.utils.transforms import unnormalize, normalize
from sklearn.metrics import confusion_matrix
from botorch.utils.multi_objective.hypervolume import Hypervolume
from botorch.utils.multi_objective.pareto import is_non_dominated
from botorch.utils.multi_objective.box_decompositions.non_dominated import (
    FastNondominatedPartitioning,
)
device = torch.device('mps')
def create_model(input_dim):
    model = nn.Sequential(
        nn.Linear(input_dim, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, 1),
        nn.Sigmoid()
    )
    return model

# set parameters
fairness_notion = 'stat-parity' #'ate' 'stat_parity'
num_clients = 3
dataset_name = 'kdd-age'
epochs = 15 #client training epochs
communication_rounds = 50

if dataset_name == 'adult':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/adult.csv'
    sensitive_feature = 'sex' #'sex': 0 ->female, 1-> male
elif dataset_name == 'adult-age':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/adult.csv'
    sensitive_feature = 'sex' #'sex': 0 ->female, 1-> male
elif dataset_name == 'bank':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/bank-full.csv'
    sensitive_feature = 'marital' #'marital': 0->married, 1-> single
elif dataset_name == 'bank-age':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/bank-full.csv'
    sensitive_feature = 'marital' #'marital': 0->married, 1-> single
elif dataset_name == 'default':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/default.csv'
    sensitive_feature = 'SEX' #'sex': 0 ->female, 1-> male
elif dataset_name == 'default-age':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/default.csv'
    sensitive_feature = 'SEX' #'sex': 0 ->female, 1-> male
elif dataset_name == 'law':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/law.csv'
    sensitive_feature = 'sex' #'sex': 0 ->female, 1-> male
elif dataset_name == 'kdd':
    url = './kdd.csv'
    sensitive_feature = 'sex' #'sex': 0 ->female, 1-> male
elif dataset_name == 'kdd-age':
    url = './kdd.csv'
    sensitive_feature = 'sex' #'sex': 0 ->female, 1-> male
elif dataset_name == 'law-income':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/law.csv'
    sensitive_feature = 'sex' #'sex': 0 ->female, 1-> male
else:
    print("dataset not supported, please update file load_data.py")
    exit()

bal_acc_list = []
fairness_notion_list = []
clients_data,X_test, y_test, sex_list, column_names_list, ytest_potential = load_dataset(url,dataset_name, num_clients, sensitive_feature)
X_test = X_test.to(device)
y_test = y_test.to(device)
global_model = create_model(X_test.shape[1])
global_model = global_model.to(device)

# Define a loss function and optimizer
criterion = nn.BCELoss()

def initialize_model(train_x, train_y):
    models = []
    for i in range(train_y.shape[-1]):
        train_objective = train_y[:,i]
        print("bismillah")
        models.append(
          SingleTaskGP(train_x, train_objective.unsqueeze(-1))
        )
    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
    return model,mll


cost_false_negatives = 10.0 #15 for adult-age
cost_false_positives = 1.0

def calculate_weights(targets, cost_false_negatives=5):
    cost_false_negatives = 10
    # Give higher weight to the positive samples because false negatives cost more
    return torch.where(targets == 1, cost_false_negatives, cost_false_positives)

def evaluate(alpha = 100):
    lr = 0.001
    # Initialize a list to store the parameters of each model
    params = [torch.zeros_like(param.data) for param in global_model.parameters()]
    for client_name in clients_data.keys():
        print(client_name)
        X1,y1,s1,y1_potential = get_data(client_name, clients_data)
        X1 = X1.to(device)
        y1 = y1.to(device)
        y1_potential = y1_potential.to(device)
        s1 = s1.to(device)
        model1 = create_model(X1.shape[1])
        model1 = model1.to(device)
        model1.load_state_dict(global_model.state_dict())
        optimizer1 = optim.Adam(model1.parameters(), lr=lr)
        if fairness_notion == 'stat_parity':
            dp_loss = DemographicParityLoss(alpha=alpha)
        elif fairness_notion == 'ate':
            dp_loss = EqualOpportunityLoss(alpha=alpha)
        #dp_loss = AverageTreatmentEffectLoss(alpha=alpha)
        for epoch in range(epochs):
            criterion = nn.BCEWithLogitsLoss(pos_weight=None)
            # Training on Client 1
            optimizer1.zero_grad()
            y_pred = model1(X1)
            #class_weights = find_class_weights(y1)
            #criterion = nn.BCELoss(weight=class_weights)
            weights = calculate_weights(y1,cost_false_negatives)
            criterion = torch.nn.BCEWithLogitsLoss(pos_weight=weights)
            X1_cpu = X1.cpu()
            X1_dataframe = pd.DataFrame(X1_cpu.numpy(), columns=column_names_list)
            y_pred_numpy = y_pred.clone().cpu()
            
            #y1_potential = find_potential_outcomes(X1_dataframe,y_pred_numpy.round().detach().numpy())
            #y1_potential = torch.tensor(y1_potential, dtype=torch.float32)
            fairness_loss = dp_loss(X1, y_pred, s1,y1_potential)
            fairness_loss = fairness_loss.to(device)
            loss = criterion(y_pred.view(-1), y1) + fairness_loss
            loss.backward()
            optimizer1.step()
        print(f'- Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')
        # After training, add the trained model's parameters to the list
        
        for param, param_sum in zip(model1.parameters(), params):
            param_sum.add_(param.data)

    # Compute average of parameters and update global model
    average_params = [param_sum / len(clients_data) for param_sum in params]
    # Copy the averaged parameters into the global model
    with torch.no_grad():
        for param_global, param_avg in zip(global_model.parameters(), average_params):
            param_global.copy_(param_avg)
    global_model.eval()
    
    # Average the model parameters (weights and biases) and set the averaged parameters to both models
    with torch.no_grad():
        y_pred = global_model(X_test).squeeze()
        y_pred_cls = y_pred.round()
        sensitivity,specificity,bal_acc,G_mean,FN_rate,FP_rate,Precision,f1_sc, acc, auc = all_metrics(y_test.cpu(),y_pred.cpu())
        stat_parity = find_statistical_parity_score(sex_list, y_test,y_pred_cls)
        X_test_cpu = X_test.cpu()
        Xtest_dataframe = pd.DataFrame(X_test_cpu.numpy(), columns=column_names_list)
        y_pred_numpy = y_pred.clone().cpu()
        #ytest_potential = find_potential_outcomes(Xtest_dataframe,y_pred_numpy.round().detach().numpy())
        ate = find_ate_2(y_pred_cls.cpu(), ytest_potential, sex_list)#0 means female-protected attribute
        #acc = (y_pred_cls == y_test).float().mean()
        auprc = average_precision_score(y_test.cpu(), y_pred.cpu())
        print(f'Communication round {round+1}/{communication_rounds}')
        if communication_rounds % 1 == 0:
            print(f'Test accuracy: {acc.item()}')
            print("sensitivity: %s" % sensitivity)
            print("specificity: %s" % specificity)
            print("BalanceACC: %s" % bal_acc)
            print("G_mean: %s" % G_mean)
            print("FN_rate: %s" % FN_rate)
            print("FP_rate: %s" % FP_rate)
            print("Precision: %s" % Precision)
            print("f1_sc: %s" % f1_sc)
            print("statistical parity: %s" % stat_parity)
            print("ate: %s" % ate)
            print("auc: %s" % auc)
            print("auprc: %s" % auprc)
    if fairness_notion == 'stat_parity':
        objectives = torch.tensor([[-stat_parity, bal_acc]]) #the two objectives
    elif fairness_notion == 'ate':
        objectives = torch.tensor([[-ate, bal_acc]]) #the two objectives
    return objectives

def models_have_same_parameters(model1, model2):
    params1 = list(model1.parameters())
    params2 = list(model2.parameters())
    print(params1)
    print("bismillah")
    print(params2)
    if len(params1) != len(params2):
        return False
    
    for p1, p2 in zip(params1, params2):
        if not torch.allclose(p1, p2):
            return False

    return True


# Train the model on both clients

for round in range(communication_rounds):
    
    print(f'Communication round {round+1}/{communication_rounds}')

    bounds = torch.tensor([[100.0], [2000.0]])  # bounds on learning rate
    #bounds = torch.tensor([[0.0], [350.0]])
    alpha = torch.tensor([100], dtype=torch.float32)
    alpha = alpha.view(1, -1)

    objectives = evaluate(alpha)
    fairness_notion_list.append(objectives[0,0].item())
    bal_acc_list.append(objectives[0,1].item())
    
    #alpha = normalize(alpha, bounds)
    x_input =  torch.tensor([100], dtype=torch.float32)#input to the optimization process
    x_input = x_input.view(1, -1)
    #model, mll = initialize_model(alpha, objectives.float())
    model, mll = initialize_model(x_input, objectives.float())

    for i in range(10):  # number of rounds of optimization
        print("Global optimization round:", i)
        fit_gpytorch_mll(mll)
        ref_point=torch.tensor([0.0001, 0.001])
        acq_func = qExpectedHypervolumeImprovement(
            model=model.float(),
            ref_point=torch.tensor([0.001, 0.001]),#problem.ref_point.tolist(),  # use known reference point
            sampler=SobolQMCNormalSampler(sample_shape=torch.Size([128])),
            # define an objective that specifies which outcomes are the objectives
            objective=IdentityMCMultiOutputObjective(outcomes=[0, 1]),
            partitioning = FastNondominatedPartitioning(ref_point, Y = objectives)
            
            # specify that the constraint is on the last outcome
            #constraints=[lambda Z: Z[..., -1]],
        )
        candidate, acq_value = optimize_acqf(
            acq_function=acq_func,
            bounds=bounds,
            q=1,
            num_restarts=300,
            raw_samples=1024,
            options ={"batch_limit": 5, "maxiter": 200}
        )


        #new_candidate = unnormalize(candidate, bounds)
        for i, m in enumerate(model.models):
          # Iterate over models and update the train data for each

            new_objectives = evaluate(candidate.item())#evaluate(candidate.item())

            for i, m in enumerate(model.models):
                train_x = torch.cat([m.train_inputs[0], candidate])
                print("bismillah")
                train_y = torch.cat([m.train_targets, new_objectives[:,i]])
                m.set_train_data(train_x, train_y, strict=False)

    # Now both models have the same parameters and we can continue with the next round of communication


global_model.eval()
    
# Average the model parameters (weights and biases) and set the averaged parameters to both models
with torch.no_grad():
        y_pred = global_model(X_test).squeeze()
        y_pred_cls = y_pred.round()
        sensitivity,specificity,bal_acc,G_mean,FN_rate,FP_rate,Precision,f1_sc, acc, auc = all_metrics(y_test.cpu(),y_pred.cpu())
        stat_parity = find_statistical_parity_score(sex_list, y_test,y_pred_cls)
        X_test_cpu = X_test.cpu()
        Xtest_dataframe = pd.DataFrame(X_test_cpu.numpy(), columns=column_names_list)
        y_pred_numpy = y_pred.clone().cpu()
        #ytest_potential = find_potential_outcomes(Xtest_dataframe,y_pred_numpy.round().detach().numpy())
        ate = find_ate_2(y_pred_cls.cpu(), ytest_potential, sex_list)#0 means female-protected attribute
        #acc = (y_pred_cls == y_test).float().mean()
        auprc = average_precision_score(y_test.cpu(), y_pred.cpu())
        print(f'Test accuracy: {acc.item()}')
        print("sensitivity: %s" % sensitivity)
        print("specificity: %s" % specificity)
        print("BalanceACC: %s" % bal_acc)
        print("G_mean: %s" % G_mean)
        print("FN_rate: %s" % FN_rate)
        print("FP_rate: %s" % FP_rate)
        print("Precision: %s" % Precision)
        print("f1_sc: %s" % f1_sc)
        print("statistical parity: %s" % stat_parity)
        print("ate: %s" % ate)
        print("auc: %s" % auc)
        print("auprc: %s" % auprc)

'''
destination = './results/default/'

if dataset_name == 'adult' or dataset_name == 'bank' or dataset_name == 'law'or dataset_name == 'default'or dataset_name == 'kdd':
    if fairness_notion == 'stat_parity':
        np.save(destination+str(num_clients)+'_bal_acc_stat_parity.npy', np.array(bal_acc_list))
        np.save(destination+str(num_clients)+'_stat_parity.npy', np.array(fairness_notion_list))
    else:
        np.save(destination+str(num_clients)+'_bal_acc_ate.npy', np.array(bal_acc_list))
        np.save(destination+str(num_clients)+'_ate.npy', np.array(fairness_notion_list))
else:
    if fairness_notion == 'stat_parity':
        np.save(destination+str(num_clients)+'_attr_bal_acc_stat_parity.npy', np.array(bal_acc_list))
        np.save(destination+str(num_clients)+'_attr_stat_parity.npy', np.array(fairness_notion_list))
    else:
        np.save(destination+str(num_clients)+'_attr_bal_acc_ate.npy', np.array(bal_acc_list))
        np.save(destination+str(num_clients)+'_attr_ate.npy', np.array(fairness_notion_list))
'''


/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/psmpy/psmpy.py:363: UserWarning: Some values do not have a match. These are dropped for purposes of establishing a matched dataframe, and subsequent calculations and plots (effect size). If you do not wish this to be the case please set drop_unmatched=False
  warnings.warn('Some values do not have a match. These are dropped for purposes of establishing a matched dataframe, and subsequent calculations and plots (effect size). If you do not wish this to be the case please set drop_unmatched=False')


Communication round 1/50
client_1
- Epoch 15/15, Loss: 1.7655870914459229
client_2
- Epoch 15/15, Loss: 1.6826237440109253
client_3
- Epoch 15/15, Loss: 1.7867023944854736
Communication round 1/50
Test accuracy: 0.20459846717760746
sensitivity: 1.0
specificity: 0.0
BalanceACC: 0.5
G_mean: 0.0
FN_rate: 0.0
FP_rate: 1.0
Precision: 0.20459846717760746
f1_sc: 0.33969571230982015
statistical parity: 0.0
ate: 0.006584769965342324
auc: 0.5448677622682035
auprc: 0.22380355855260708
bismillah
bismillah
Global optimization round: 0


/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/botorch/models/gpytorch.py:129: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  warnings.warn(_get_single_precision_warning(X.dtype), UserWarning)
/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/botorch/models/utils/assorted.py:173: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)
/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/botorch/models/utils/assorted.py:201: InputDataWarning: Input data is not standardized. Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


client_1
- Epoch 15/15, Loss: 1.7126370668411255
client_2
- Epoch 15/15, Loss: 1.6482796669006348
client_3
- Epoch 15/15, Loss: 1.7352889776229858
Communication round 1/50
Test accuracy: 0.34121959346884373
sensitivity: 0.9153094462540716
specificity: 0.1935483870967742
BalanceACC: 0.5544289166754229
G_mean: 0.4208998301459827
FN_rate: 0.08469055374592833
FP_rate: 0.8064516129032258
Precision: 0.2259750703659027
f1_sc: 0.36246372138019994
statistical parity: -0.005871257195555479
ate: 0.006032116155008926
auc: 0.6955628956522094
auprc: 0.44067556062718616
bismillah
bismillah
client_1
- Epoch 15/15, Loss: 1.6948330402374268
client_2
- Epoch 15/15, Loss: 1.6300936937332153
client_3
- Epoch 15/15, Loss: 1.7203803062438965
Communication round 1/50
Test accuracy: 0.39186937687437523
sensitivity: 0.8794788273615635
specificity: 0.2664432341851697
BalanceACC: 0.5729610307733666
G_mean: 0.48407766232247845
FN_rate: 0.12052117263843648
FP_rate: 0.7335567658148303
Precision: 0.2357049323439546
f

client_1
- Epoch 15/15, Loss: 1.6378138065338135
client_2
- Epoch 15/15, Loss: 1.5679361820220947
client_3
- Epoch 15/15, Loss: 1.6659036874771118
Communication round 1/50
Test accuracy: 0.5568143952015995
sensitivity: 0.8469055374592834
specificity: 0.482195224130708
BalanceACC: 0.6645503807949957
G_mean: 0.6390413174848062
FN_rate: 0.15309446254071662
FP_rate: 0.517804775869292
Precision: 0.296127562642369
f1_sc: 0.4388185654008439
statistical parity: 0.004458641360127302
ate: 0.007753198872261974
auc: 0.7617012072722906
auprc: 0.4734569716098088
bismillah
bismillah
client_1
- Epoch 15/15, Loss: 1.6361191272735596
client_2
- Epoch 15/15, Loss: 1.566960096359253
client_3
- Epoch 15/15, Loss: 1.6644196510314941
Communication round 1/50
Test accuracy: 0.5558147284238587
sensitivity: 0.8517915309446255
specificity: 0.4796816087138668
BalanceACC: 0.6657365698292461
G_mean: 0.6392094585129082
FN_rate: 0.1482084690553746
FP_rate: 0.5203183912861332
Precision: 0.2963172804532578
f1_sc: 0.439

...

/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/botorch/optim/initializers.py:403: BadInitialCandidatesWarning: Unable to find non-zero acquisition function values - initial conditions are being selected randomly.
  warnings.warn(


client_1
- Epoch 15/15, Loss: 1.5728636980056763
client_2
- Epoch 15/15, Loss: 1.49663507938385
client_3
- Epoch 15/15, Loss: 1.5871647596359253
Communication round 50/50
Test accuracy: 0.6334555148283906
sensitivity: 0.7980456026058632
specificity: 0.5911185588604944
BalanceACC: 0.6945820807331788
G_mean: 0.686832997545497
FN_rate: 0.20195439739413681
FP_rate: 0.40888144113950564
Precision: 0.33424283765347884
f1_sc: 0.47115384615384615
statistical parity: 0.0440964836603252
ate: 0.016814769419335074
auc: 0.7584895927861149
auprc: 0.40388772731528777
bismillah
bismillah
client_1
- Epoch 15/15, Loss: 1.57317316532135
client_2
- Epoch 15/15, Loss: 1.494706392288208
client_3
- Epoch 15/15, Loss: 1.586754560470581
Communication round 50/50
Test accuracy: 0.6331222925691437
sensitivity: 0.7964169381107492
specificity: 0.5911185588604944
BalanceACC: 0.6937677484856217
G_mean: 0.6861317896061322
FN_rate: 0.20358306188925082
FP_rate: 0.40888144113950564
Precision: 0.3337883959044369
f1_sc: 0.

/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/botorch/optim/initializers.py:403: BadInitialCandidatesWarning: Unable to find non-zero acquisition function values - initial conditions are being selected randomly.
  warnings.warn(


client_1
- Epoch 15/15, Loss: 1.574999213218689
client_2
- Epoch 15/15, Loss: 1.4959410429000854
client_3
- Epoch 15/15, Loss: 1.5867382287979126
Communication round 50/50
Test accuracy: 0.6367877374208597
sensitivity: 0.7931596091205212
specificity: 0.5965647255969837
BalanceACC: 0.6948621673587525
G_mean: 0.6878742941625269
FN_rate: 0.20684039087947884
FP_rate: 0.40343527440301635
Precision: 0.33586206896551724
f1_sc: 0.4718992248062015
statistical parity: 0.03616277945529456
ate: 0.021091431532337834
auc: 0.7580334712046386
auprc: 0.4027563373117368
bismillah
bismillah
client_1
- Epoch 15/15, Loss: 1.5755363702774048
client_2
- Epoch 15/15, Loss: 1.4953058958053589
client_3
- Epoch 15/15, Loss: 1.586125135421753
Communication round 50/50
Test accuracy: 0.6394535154948351
sensitivity: 0.7866449511400652
specificity: 0.6015919564306661
BalanceACC: 0.6941184537853656
G_mean: 0.6879238876304976
FN_rate: 0.21335504885993486
FP_rate: 0.3984080435693339
Precision: 0.3368200836820084
f1_sc:

"\ndestination = './results/default/'\n\nif dataset_name == 'adult' or dataset_name == 'bank' or dataset_name == 'law'or dataset_name == 'default'or dataset_name == 'kdd':\n    if fairness_notion == 'stat_parity':\n        np.save(destination+str(num_clients)+'_bal_acc_stat_parity.npy', np.array(bal_acc_list))\n        np.save(destination+str(num_clients)+'_stat_parity.npy', np.array(fairness_notion_list))\n    else:\n        np.save(destination+str(num_clients)+'_bal_acc_ate.npy', np.array(bal_acc_list))\n        np.save(destination+str(num_clients)+'_ate.npy', np.array(fairness_notion_list))\nelse:\n    if fairness_notion == 'stat_parity':\n        np.save(destination+str(num_clients)+'_attr_bal_acc_stat_parity.npy', np.array(bal_acc_list))\n        np.save(destination+str(num_clients)+'_attr_stat_parity.npy', np.array(fairness_notion_list))\n    else:\n        np.save(destination+str(num_clients)+'_attr_bal_acc_ate.npy', np.array(bal_acc_list))\n        np.save(destination+str(num_c

In [2]:
destination = './results/default/'

if dataset_name == 'adult' or dataset_name == 'bank' or dataset_name == 'law'or dataset_name == 'default'or dataset_name == 'kdd':
    if fairness_notion == 'stat_parity':
        np.save(destination+str(num_clients)+'_bal_acc_stat_parity.npy', np.array(bal_acc_list))
        np.save(destination+str(num_clients)+'_stat_parity.npy', np.array(fairness_notion_list))
    else:
        np.save(destination+str(num_clients)+'_bal_acc_ate.npy', np.array(bal_acc_list))
        np.save(destination+str(num_clients)+'_ate.npy', np.array(fairness_notion_list))
else:
    if fairness_notion == 'stat_parity':
        np.save(destination+str(num_clients)+'_attr_bal_acc_stat_parity.npy', np.array(bal_acc_list))
        np.save(destination+str(num_clients)+'_attr_stat_parity.npy', np.array(fairness_notion_list))
    else:
        np.save(destination+str(num_clients)+'_attr_bal_acc_ate.npy', np.array(bal_acc_list))
        np.save(destination+str(num_clients)+'_attr_ate.npy', np.array(fairness_notion_list))

In [ ]:
#final bototrch model

##federated learning with 2 clients

import torch
import math
#import sys

#sys.path.append('/content/drive/My Drive')
from utilities import find_ate, find_ate_2, find_statistical_parity_score, find_eqop_score, all_metrics
from load_data_redefined_ate import get_data, load_dataset
from constraint import AverageTreatmentEffectLoss, DemographicParityLoss, EqualOpportunityLoss
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score

from torch import nn
from torch import optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

from torch.utils.data import DataLoader
from torch import nn, optim
from torch.utils.data import TensorDataset
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from botorch.models import SingleTaskGP, ModelListGP
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch.fit import fit_gpytorch_model
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import qExpectedImprovement
from botorch.optim import optimize_acqf
from itertools import chain
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.acquisition.multi_objective.monte_carlo import (
    qExpectedHypervolumeImprovement,
    qNoisyExpectedHypervolumeImprovement,
)
from botorch.acquisition.multi_objective.objective import IdentityMCMultiOutputObjective
from botorch import fit_gpytorch_mll
import math
from botorch.utils.transforms import unnormalize, normalize
from sklearn.metrics import confusion_matrix
from botorch.utils.multi_objective.hypervolume import Hypervolume
from botorch.utils.multi_objective.pareto import is_non_dominated
from botorch.utils.multi_objective.box_decompositions.non_dominated import (
    FastNondominatedPartitioning,
)
device = torch.device('mps')
def create_model(input_dim):
    model = nn.Sequential(
        nn.Linear(input_dim, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, 1),
        nn.Sigmoid()
    )
    return model

# set parameters
fairness_notion = 'stat_parity' #'ate' 'stat_parity'
num_clients = 3
dataset_name = 'adult-age'
epochs = 15 #client training epochs
communication_rounds = 50

if dataset_name == 'adult':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/adult.csv'
    sensitive_feature = 'sex' #'sex': 0 ->female, 1-> male
elif dataset_name == 'adult-age':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/adult.csv'
    sensitive_feature = 'sex' #'sex': 0 ->female, 1-> male
elif dataset_name == 'bank':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/bank-full.csv'
    sensitive_feature = 'marital' #'marital': 0->married, 1-> single
elif dataset_name == 'bank-age':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/bank-full.csv'
    sensitive_feature = 'marital' #'marital': 0->married, 1-> single
elif dataset_name == 'default':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/default.csv'
    sensitive_feature = 'SEX' #'sex': 0 ->female, 1-> male
elif dataset_name == 'default-age':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/default.csv'
    sensitive_feature = 'SEX' #'sex': 0 ->female, 1-> male
elif dataset_name == 'law':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/law.csv'
    sensitive_feature = 'sex' #'sex': 0 ->female, 1-> male
elif dataset_name == 'kdd':
    url = './kdd.csv'
    sensitive_feature = 'sex' #'sex': 0 ->female, 1-> male
elif dataset_name == 'kdd-age':
    url = './kdd.csv'
    sensitive_feature = 'sex' #'sex': 0 ->female, 1-> male
elif dataset_name == 'law-income':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/law.csv'
    sensitive_feature = 'sex' #'sex': 0 ->female, 1-> male
    

bal_acc_list = []
fairness_notion_list = []
clients_data,X_test, y_test, sex_list, column_names_list, ytest_potential = load_dataset(url,dataset_name, num_clients, sensitive_feature)
X_test = X_test.to(device)
y_test = y_test.to(device)
global_model = create_model(X_test.shape[1])
global_model = global_model.to(device)

# Define a loss function and optimizer
criterion = nn.BCELoss()

def initialize_model(train_x, train_y):
    models = []
    for i in range(train_y.shape[-1]):
        train_objective = train_y[:,i]
        print("bismillah")
        models.append(
          SingleTaskGP(train_x, train_objective.unsqueeze(-1))
        )
    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
    return model,mll


cost_false_negatives = 10.0 #15 for adult-age
cost_false_positives = 1.0

def calculate_weights(targets, cost_false_negatives=5):
    cost_false_negatives = 10
    # Give higher weight to the positive samples because false negatives cost more
    return torch.where(targets == 1, cost_false_negatives, cost_false_positives)

def evaluate(alpha = 100, lr=0.001, cost_false_negatives=5):
    # Initialize a list to store the parameters of each model
    params = [torch.zeros_like(param.data) for param in global_model.parameters()]
    for client_name in clients_data.keys():
        print(client_name)
        X1,y1,s1,y1_potential = get_data(client_name, clients_data)
        X1 = X1.to(device)
        y1 = y1.to(device)
        y1_potential = y1_potential.to(device)
        s1 = s1.to(device)
        model1 = create_model(X1.shape[1])
        model1 = model1.to(device)
        model1.load_state_dict(global_model.state_dict())
        optimizer1 = optim.Adam(model1.parameters(), lr=lr)
        if fairness_notion == 'stat_parity':
            dp_loss = DemographicParityLoss(alpha=alpha)
        elif fairness_notion == 'ate':
            dp_loss = EqualOpportunityLoss(alpha=alpha)
        #dp_loss = AverageTreatmentEffectLoss(alpha=alpha)
        for epoch in range(epochs):
            criterion = nn.BCEWithLogitsLoss(pos_weight=None)
            # Training on Client 1
            optimizer1.zero_grad()
            y_pred = model1(X1)
            #class_weights = find_class_weights(y1)
            #criterion = nn.BCELoss(weight=class_weights)
            weights = calculate_weights(y1,cost_false_negatives)
            criterion = torch.nn.BCEWithLogitsLoss(pos_weight=weights)
            X1_cpu = X1.cpu()
            X1_dataframe = pd.DataFrame(X1_cpu.numpy(), columns=column_names_list)
            y_pred_numpy = y_pred.clone().cpu()
            
            #y1_potential = find_potential_outcomes(X1_dataframe,y_pred_numpy.round().detach().numpy())
            #y1_potential = torch.tensor(y1_potential, dtype=torch.float32)
            fairness_loss = dp_loss(X1, y_pred, s1,y1_potential)
            fairness_loss = fairness_loss.to(device)
            loss = criterion(y_pred.view(-1), y1) + fairness_loss
            loss.backward()
            optimizer1.step()
        print(f'- Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')
        # After training, add the trained model's parameters to the list
        
        for param, param_sum in zip(model1.parameters(), params):
            param_sum.add_(param.data)

    # Compute average of parameters and update global model
    average_params = [param_sum / len(clients_data) for param_sum in params]
    # Copy the averaged parameters into the global model
    with torch.no_grad():
        for param_global, param_avg in zip(global_model.parameters(), average_params):
            param_global.copy_(param_avg)
    global_model.eval()
    
    # Average the model parameters (weights and biases) and set the averaged parameters to both models
    with torch.no_grad():
        y_pred = global_model(X_test).squeeze()
        y_pred_cls = y_pred.round()
        sensitivity,specificity,bal_acc,G_mean,FN_rate,FP_rate,Precision,f1_sc, acc, auc = all_metrics(y_test.cpu(),y_pred.cpu())
        stat_parity = find_statistical_parity_score(sex_list, y_test,y_pred_cls)
        X_test_cpu = X_test.cpu()
        Xtest_dataframe = pd.DataFrame(X_test_cpu.numpy(), columns=column_names_list)
        y_pred_numpy = y_pred.clone().cpu()
        #ytest_potential = find_potential_outcomes(Xtest_dataframe,y_pred_numpy.round().detach().numpy())
        ate = find_ate_2(y_pred_cls.cpu(), ytest_potential, sex_list)#0 means female-protected attribute
        #acc = (y_pred_cls == y_test).float().mean()
        auprc = average_precision_score(y_test.cpu(), y_pred.cpu())
        print(f'Communication round {round+1}/{communication_rounds}')
        if communication_rounds % 1 == 0:
            print(f'Test accuracy: {acc.item()}')
            print("sensitivity: %s" % sensitivity)
            print("specificity: %s" % specificity)
            print("BalanceACC: %s" % bal_acc)
            print("G_mean: %s" % G_mean)
            print("FN_rate: %s" % FN_rate)
            print("FP_rate: %s" % FP_rate)
            print("Precision: %s" % Precision)
            print("f1_sc: %s" % f1_sc)
            print("statistical parity: %s" % stat_parity)
            print("ate: %s" % ate)
            print("auc: %s" % auc)
            print("auprc: %s" % auprc)
    if fairness_notion == 'stat_parity':
        objectives = torch.tensor([[-stat_parity, bal_acc]]) #the two objectives
    elif fairness_notion == 'ate':
        objectives = torch.tensor([[-ate, bal_acc]]) #the two objectives
    return objectives

def models_have_same_parameters(model1, model2):
    params1 = list(model1.parameters())
    params2 = list(model2.parameters())
    print(params1)
    print("bismillah")
    print(params2)
    if len(params1) != len(params2):
        return False
    
    for p1, p2 in zip(params1, params2):
        if not torch.allclose(p1, p2):
            return False

    return True


# Train the model on both clients

for round in range(communication_rounds):
    
    print(f'Communication round {round+1}/{communication_rounds}')

    bounds = torch.tensor([[100.0,0.0,5], [1000.0,0.01,30]])  # bounds on learning rate
    #bounds = torch.tensor([[0.0], [350.0]])
    alpha = torch.tensor([100], dtype=torch.float32)
    alpha = alpha.view(1, -1)

    objectives = evaluate(alpha)
    fairness_notion_list.append(objectives[0,0].item())
    bal_acc_list.append(objectives[0,1].item())
    
    #alpha = normalize(alpha, bounds)
    x_input =  torch.tensor([100,0.001,5], dtype=torch.float32)#input to the optimization process
    x_input = x_input.view(1, -1)
    #model, mll = initialize_model(alpha, objectives.float())
    model, mll = initialize_model(x_input, objectives.float())

    for i in range(10):  # number of rounds of optimization
        print("Global optimization round:", i)
        fit_gpytorch_mll(mll)
        ref_point=torch.tensor([0.0001, 0.001])
        acq_func = qExpectedHypervolumeImprovement(
            model=model.float(),
            ref_point=torch.tensor([0.0001, 0.001]),#problem.ref_point.tolist(),  # use known reference point
            sampler=SobolQMCNormalSampler(sample_shape=torch.Size([128])),
            # define an objective that specifies which outcomes are the objectives
            objective=IdentityMCMultiOutputObjective(outcomes=[0, 1]),
            partitioning = FastNondominatedPartitioning(ref_point, Y = objectives)
            
            # specify that the constraint is on the last outcome
            #constraints=[lambda Z: Z[..., -1]],
        )
        candidate, acq_value = optimize_acqf(
            acq_function=acq_func,
            bounds=bounds,
            q=1,
            num_restarts=300,
            raw_samples=1024,
            options ={"batch_limit": 5, "maxiter": 200}
        )


        #new_candidate = unnormalize(candidate, bounds)
        for i, m in enumerate(model.models):
          # Iterate over models and update the train data for each

            new_objectives = evaluate(candidate[0,0].item(), candidate[0,1].item(), candidate[0,2].item())#evaluate(candidate.item())

            for i, m in enumerate(model.models):
                train_x = torch.cat([m.train_inputs[0], candidate])
                print("bismillah")
                train_y = torch.cat([m.train_targets, new_objectives[:,i]])
                m.set_train_data(train_x, train_y, strict=False)

    # Now both models have the same parameters and we can continue with the next round of communication


global_model.eval()
    
# Average the model parameters (weights and biases) and set the averaged parameters to both models
with torch.no_grad():
        y_pred = global_model(X_test).squeeze()
        y_pred_cls = y_pred.round()
        sensitivity,specificity,bal_acc,G_mean,FN_rate,FP_rate,Precision,f1_sc, acc, auc = all_metrics(y_test.cpu(),y_pred.cpu())
        stat_parity = find_statistical_parity_score(sex_list, y_test,y_pred_cls)
        X_test_cpu = X_test.cpu()
        Xtest_dataframe = pd.DataFrame(X_test_cpu.numpy(), columns=column_names_list)
        y_pred_numpy = y_pred.clone().cpu()
        #ytest_potential = find_potential_outcomes(Xtest_dataframe,y_pred_numpy.round().detach().numpy())
        ate = find_ate_2(y_pred_cls.cpu(), ytest_potential, sex_list)#0 means female-protected attribute
        #acc = (y_pred_cls == y_test).float().mean()
        auprc = average_precision_score(y_test.cpu(), y_pred.cpu())
        print(f'Test accuracy: {acc.item()}')
        print("sensitivity: %s" % sensitivity)
        print("specificity: %s" % specificity)
        print("BalanceACC: %s" % bal_acc)
        print("G_mean: %s" % G_mean)
        print("FN_rate: %s" % FN_rate)
        print("FP_rate: %s" % FP_rate)
        print("Precision: %s" % Precision)
        print("f1_sc: %s" % f1_sc)
        print("statistical parity: %s" % stat_parity)
        print("ate: %s" % ate)
        print("auc: %s" % auc)
        print("auprc: %s" % auprc)

destination = './results/adult/'

if fairness_notion == 'stat_parity':
    np.save(destination+str(num_clients)+'_attr_bal_acc_stat_parity.npy', np.array(bal_acc_list))
    np.save(destination+str(num_clients)+'_attr_stat_parity.npy', np.array(fairness_notion_list))
else:
    np.save(destination+str(num_clients)+'_attr_bal_acc_ate.npy', np.array(bal_acc_list))
    np.save(destination+str(num_clients)+'_attr_ate.npy', np.array(fairness_notion_list))